In [ ]:
#pip install h3pandas

In [ ]:
import pandas as pd
import altair as alt
import geopandas as gpd
import os
import h3pandas as h3

dir = '../Data'

weather = pd.read_csv(f'{dir}/weather_clean.csv')
collisions = pd.read_csv(f'{dir}/collisions_clean.csv')
nyc_map = gpd.read_file(f'{dir}/new-york-city-boroughs-ny_.geojson')


alt.data_transformers.enable("vegafusion")

collisions['YEAR'] = collisions['CRASH DATE'].astype(str).str[:4]

In [ ]:
# create a new column that sums NUMBER OF PERSONS INJURED and NUMBER OF PERSONS KILLED
collisions['TOTAL INJURED/KILLED'] = collisions['NUMBER OF INJURED'] + collisions['NUMBER OF KILLED']

collisions = collisions[collisions['LATITUDE'].notna() &  collisions['LONGITUDE'].notna() & collisions['BOROUGH'].notna() & collisions['BOROUGH'].notna()]

## Are there any areas with a larger number of accidents?

https://vega.github.io/vega/docs/schemes/

This is the map of New York City diveded in boroughs, we first thought about using a dot map. However, the great amount of instances, the limited space for the visualizations and the restriction of not using interactive plots, made us change our mind after some simple tries.

In [ ]:
nyc = alt.Chart(nyc_map).mark_geoshape(
    stroke='white',
    strokeWidth=1,
    filled=True
).encode(
    color=alt.Color('name',
                    scale=alt.Scale(scheme='greys'),
                    # scale=alt.Scale(scheme='purples'),
                    # scale=alt.Scale(scheme='tealblues'),
                    title='Borough'),
).project(
    type='identity', reflectY=True
)
nyc

In [ ]:
nyc_map_hex = nyc_map.h3.polyfill_resample(8)
nyc_map_hex = nyc_map_hex.reset_index()
nyc_map_hex.head()

In [ ]:
df = nyc_map_hex[['h3_polyfill', 'name', 'geometry']]
nyc_map_hex = nyc_map.h3.polyfill_resample(8)
nyc_map_hex = nyc_map_hex.reset_index()
nyc_map_hex.head()

We decided to do a choroleth map, but diving the main regions into smaller geometrical forms (hexagons), so the areas would be easier to compare. We use a sequential palette that represnets the quantites aggrupaetd in every hexagon, so the user can see in which areas thers a larger concentration of accidents. We decided to keep the shape of the original map to make orientation easier for the user.

In [ ]:
# PONER LOS NOMBRES DE LOS BOROUGHS

In [ ]:
# if os.path.exists(f'{dir}/nyc_hex_map.json'):
#     df = pd.read_json(f'{dir}/nyc_hex_map.json')
# else:
#     collisions_geo = gpd.GeoDataFrame(collisions, geometry=gpd.points_from_xy(collisions['LONGITUDE'], collisions['LATITUDE']))

#     collisions_geo.crs = nyc_map_hex.crs
    
#     df = collisions_geo.sjoin(nyc_map_hex, how='right')
#     # save to csv
#     df.to_json(f'{dir}/nyc_hex_map.json', orient='records', na='null')

In [ ]:
collisions_geo = gpd.GeoDataFrame(collisions, geometry=gpd.points_from_xy(collisions['LONGITUDE'], collisions['LATITUDE']))

collisions_geo.crs = nyc_map_hex.crs

df = collisions_geo.sjoin(nyc_map_hex, how='right')

tmp = df.groupby(['h3_polyfill', 'name']).count().reset_index()
tmp = tmp[['h3_polyfill', 'name', 'COLLISION_ID']]
tmp.columns = ['h3_polyfill', 'name', 'count']

In [ ]:
df = df.merge(tmp, on=['h3_polyfill', 'name'], how='left')

df = df[['h3_polyfill', 'name', 'count', 'geometry']]

df = df.drop_duplicates(subset=['h3_polyfill', 'name'])

c1 = alt.Chart(df).mark_geoshape(
    stroke='white',
    strokeWidth=1,
    filled=True
).encode(
    color=alt.Color('count:Q',
                    # scale=alt.Scale(scheme='greys'),
                    # scale=alt.Scale(scheme='tealblues'),
                    scale=alt.Scale(range=['#B3E9C7', '#5603ad']),
                    title='Borough'),
).project(
    type='identity', reflectY=True
).properties(
    width=500,
    height=500
)

c2 = alt.Chart(nyc_map).mark_geoshape(
    stroke='#1d3557',
    strokeWidth=1,
    filled=False
).project(
    type='identity', reflectY=True
).properties(
    width=500,
    height=500
)

c1 + c2

In [ ]:
nyc_map_hex.crs

In [ ]:
nyc_map.crs

In [ ]:
# select LATITUDE and LONGITUDE columns and convert them to a GeoDataFrame
collisions_geo = gpd.GeoDataFrame(collisions[['LATITUDE', 'LONGITUDE', 'BOROUGH', 'ZIP CODE', 'TOTAL INJURED/KILLED']], geometry=gpd.points_from_xy(collisions.LONGITUDE, collisions.LATITUDE))

collisions_geo.crs = "EPSG:4326"
collisions_geo.crs

In [ ]:
nyc_map_hex.head(1)

In [ ]:
df = collisions[['BOROUGH', 'CRASH DATE']].groupby('BOROUGH').count().reset_index().head()
df.columns = ['BOROUGH', 'COLLISION COUNT']

df2 = collisions[['BOROUGH', 'TOTAL INJURED/KILLED']].groupby('BOROUGH').sum().reset_index().head()

df3 = collisions[['BOROUGH', 'LONGITUDE', 'LATITUDE']].groupby('BOROUGH').mean().reset_index().head()
df3.columns = ['BOROUGH', 'MEAN LONGITUDE', 'MEAN LATITUDE']

df = df.merge(df2, on='BOROUGH', how='left')
df = df.merge(df3, on='BOROUGH', how='left')
df

ESTO NO SE QUE WEA ES PERO NO SE ME EJECUTA ASÍ QUE LO COMENTÉ

In [ ]:
circles = alt.Chart(df).mark_circle(
    tooltip=True,
    filled=True,
    opacity=1
).encode(
    x=alt.X('MEAN LONGITUDE:Q', axis=None),
    y=alt.Y('MEAN LATITUDE:Q', axis=None),
    size=alt.Size('COLLISION COUNT:Q',
                  title='Collision Count')
).properties(
    title='Number of Collisions by Borough'
)

circles2 = alt.Chart().mark_circle(
    tooltip=True,
    filled=True,
    opacity=1
).encode(
    x=alt.X('MEAN LONGITUDE:Q', axis=None),
    y=alt.Y('MEAN LATITUDE:Q', axis=None),
    size=alt.Size('TOTAL INJURED/KILLED:Q',
                  title='Total Injured/Killed')
).properties(
    title='Number of Injured/Killed by Borough'
)

nyc + circles | nyc + circles2

In [ ]:
population = pd.DataFrame({
    'BOROUGH': ['BRONX', 'BROOKLYN', 'MANHATTAN', 'QUEENS', 'STATEN ISLAND'],
    'POPULATION_2018': [1438000000, 2601000000, 1632000000, 2299000000, 474101],
    'POPULATION_2020': [1427000000, 2577000000, 1629000000, 2271000000, 475596],
    'CAR OWNERSHIP': [0.40, 0.44, 0.22, 0.62, 0.83]
})

population['MEAN POPULATION'] = population[['POPULATION_2018', 'POPULATION_2020']].mean(axis=1)

population

In [ ]:
total_population = population['MEAN POPULATION'].sum()

df = collisions[['BOROUGH', 'CRASH TIME INTERVAL']]
df.insert(0, 'COUNT', 1)

df = df.groupby(['BOROUGH', 'CRASH TIME INTERVAL']).count().reset_index()
df = df.merge(population[['BOROUGH', 'MEAN POPULATION', 'CAR OWNERSHIP']], on='BOROUGH', how='left')

df['NORMALIZED COUNT'] = df['COUNT'] * df['CAR OWNERSHIP'] 

df.describe()

Although we consider the map to be a pretty good option, we also thought that a linechart representing the normalized count of accidents along the day hours for each borough could also be a good option to answer the proposed question. We normalized the count of accidents with the information of the follwoing [article](https://edc.nyc/article/new-yorkers-and-their-cars), which especifies the percentage of people in every borough owns a car. This also helps give an extra view on the most common hour of the day were accidents occur.

In [ ]:
# poner grid en las X

In [ ]:
alt.Chart(df).mark_line(
    tooltip=True,
    size=2
).encode(
    x=alt.X('CRASH TIME INTERVAL:N',
            title='Crash Time Interval',
            axis=alt.Axis(labelAngle=0)),
    y=alt.Y('NORMALIZED COUNT:Q',
            title='Number of Collisions normalized by Car Ownership Data'),
    color=alt.Color('BOROUGH:N',
                    scale=alt.Scale(range=['#a3ffd6', '#d69bf5', '#ff8080', '#80ff80', '#80bfff']),
                    title='Borough')
).properties(
    title='Normalized Number of Collisions by Time Interval'
)

DECISION FINAL: USAR LOS DOS O SOLO EL MAPA, YA VEMOS